In [47]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.action_chains import ActionChains
import csv
import scrapy
import time
import re
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
from selenium.webdriver.firefox.options import Options
import multiprocessing as mp


In [38]:
#you will need to put your own path to the firefox binary here
#binary = FirefoxBinary()
legistar = 'https://milwaukee.legistar.com/Legislation.aspx'

def visit_site(legistar, headless=False, phantom=False):
    if phantom == False:
        if headless:
            options = Options()
            options.headless = True
            mkeDriver = webdriver.Firefox(options = options, executable_path='/Users/ayaspan/Documents/Personal/voting_records_mke/geckodriver')
        else:
            mkeDriver = webdriver.Firefox(executable_path='/Users/ayaspan/Documents/Personal/voting_records_mke/geckodriver')
        try: 
            mkeDriver.get(legistar)
        except:
            return None
        return mkeDriver
    else:
        mkeDriver = webdriver.PhantomJS()
        mkeDriver.get(legistar)
        return mkeDriver


In [375]:
mkeDriver2 = visit_site(legistar)

In [385]:
test = get_next_page(mkeDriver2)

[]


In [3]:
def select_all_years(mkeDriver):
    years = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstYears_Input")
    years.send_keys('All Years')
    years.submit()
    #return years

In [4]:
#makes it so that you have max results

def get_max_results(mkeDriver):
    step1 = mkeDriver.find_elements_by_id("ctl00_ContentPlaceHolder1_menuMain")[0]
    action = ActionChains(mkeDriver)
    action.click(step1.find_elements_by_class_name("rmItem ")[1])
    #action.click(step1.find_elements_by_class_name("rmItem ")[1].find_element_by_css_selector("a"))
    action.perform()
    time.sleep(1)
    step2 = step1.find_elements_by_class_name("rmItem ")[1].parent
    #print(step2.text)
    action = ActionChains(mkeDriver)
    # action_test2 = ActionChains(testDriver)
    action.move_to_element(step2.find_element_by_class_name("rmSlide").find_element_by_class_name("rmLast"))
    action.click(step2.find_element_by_class_name("rmSlide").find_element_by_class_name("rmLast"))
    action.perform()
    #return mkeDriver

In [5]:
# re.search(r'(?<=of\s)[0-9]+',mkeDriver.find_elements_by_class_name("rgPagerCell")[0]\
#             .find_element_by_class_name("rgInfoPart").text)[0]

In [6]:
#need to consider the recursion involved in looping through pages OR use above

In [7]:
#locats page link 
def get_next_page(mkeDriver):
    pgs_drop = mkeDriver.find_elements_by_class_name("rgPagerCell")
    #print(pgs_drop)
    try:
        return pgs_drop[0].find_element_by_xpath("//a[@class='rgCurrentPage']/following-sibling::a")
    except:
        return None

def go_to_next_page(next_page):
    next_page.click()

In [19]:
def select_ordinance(mkeDriver):
    type_ordinance = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstTypeBasic_Arrow").click()
    legislation_type = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstTypeBasic_DropDown")\
                .find_elements_by_class_name("rcbItem")
    print(legislation_type[15].text)
    actions = ActionChains(mkeDriver)
    actions.move_to_element(legislation_type[15])
    actions.click(legislation_type[15])
    actions.perform()
    #choose_ordinance.perform()#.double_click()
#type_legislation.s
#type_legislation.submit()

In [9]:
# legislation_type = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstTypeBasic_DropDown")\
#                 .find_elements_by_class_name("rcbItem")
# hover_ord = ActionChains(mkeDriver).move_to_element(legislation_type[14])
# hover_ord.perform().send_keys("Ordinance").double_click()

In [10]:
def start_search(mkeDriver):
    s_button = mkeDriver.find_element_by_id("visibleSearchButton")
    s_button.click()
    #return mkeDriver

In [64]:
def get_records_table(mkeDriver):
    records_table = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_gridMain")
    r_table_body = records_table.find_elements_by_class_name("rgRow")
    r_table_body2 = records_table.find_elements_by_class_name("rgAltRow")
    return r_table_body, r_table_body2

In [12]:
# for i in records_table.find_elements_by_css_selector('a'):
#     if 'https' in i.get_attribute('href'):
#         print(i.get_attribute('href'))

In [13]:
#r_table_body = records_table.find_element_by_css_selector("tbody")

In [14]:
#can change # of records per page :)

In [63]:
#if status passed
import time

def get_meta_data(legistar):
    count=0
    leg_dict = {}
    col_names = [0,'type','status','created_date','final_action','title']
    
    #CALL OTHER REQUISITE FUNCTIONS FIRST
    mkeDriver = visit_site(legistar)
    #time.sleep(5)
    select_all_years(mkeDriver)
    time.sleep(2)
    select_ordinance(mkeDriver)
    start_search(mkeDriver)
    time.sleep(2)
    get_max_results(mkeDriver)
    
    while True:
        time.sleep(4)
        r_table_body, r_table_body2 = get_records_table(mkeDriver)
        if count in leg_dict:
            print('dup')
            continue
        else:
            leg_dict[count]= {i.find_element_by_css_selector('td').find_element_by_css_selector('a').text:{
                          'link': i.find_element_by_css_selector('a').get_attribute('href'),
                           'other_info':[j.text for j in i.find_elements_by_css_selector('td')]} for i in r_table_body}
            count+=1
            leg_dict[count] = {i.find_element_by_css_selector('td').find_element_by_css_selector('a').text:{
                          'link': i.find_element_by_css_selector('a').get_attribute('href'),
                           'other_info':[j.text for j in i.find_elements_by_css_selector('td')]} for i in r_table_body2}
        #works
        #leg_dict[count] = [i.text for i in r_table_body]
        
        #print(r_table_body.text)
        #print(r_table_body.find_elements_by_class_name('rgRow')[0].text)
#         for row in r_table_body.find_elements_by_class_name('rgRow'):
#             print(row.get_attribute('id'),'id')
#             file_num = row.find_element_by_css_selector('td').find_element_by_css_selector('a').text
#             leg_dict[file_num] = {}
#             link = row.find_element_by_id('ctl00_ContentPlaceHolder1_gridMain_ctl00_ctl04_hypFile').get_attribute('href')
#             leg_dict[file_num]['file_link'] = link
#             print(file_num,'file_num and ', link,' link')

#             for d,col in zip(row.find_elements_by_css_selector('td'),col_names):
#                 if not col:
#                     continue
#                 #if leg_dict[file_num]['status'] in ('Passed',"Mayor's Office",'Dead','Placed On File'): use this later for going through links
#                 leg_dict[file_num][col] = d.text
# #                 leg_dict[file_num]['status'] = d.text
# #                 leg_dict[file_num]['created_date'] = d.text
# #                 leg_dict[file_num]['final_action'] = d.text
#                 leg_dict[file_num]['title'] = d.text
        time.sleep(.5)
        
        
        next_page = get_next_page(mkeDriver)
        
        if not next_page:
            return leg_dict
        else:
            next_page.click()
        
        
        count+=1
    

In [60]:
#leg_dict

In [65]:
meta_legis_dict = get_meta_data(legistar)

Ordinance


In [68]:
df = pd.DataFrame()

for key,val in meta_legis_dict.items():
    df = df.append(pd.DataFrame.from_dict(val).T)


In [69]:
df.shape

(4716, 2)

In [120]:
def splitDataFrameList(df,target_column,col_list):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    row_accumulator = []

    def splitListToRows(row):
        #new_row = {}
        split_row = row[target_column]
        new_row = row.to_dict()
        for col_name, s in zip(col_list,split_row):
            new_row[col_name] = s
        row_accumulator.append(new_row)

    df.apply(splitListToRows, axis=1)
    new_df = pd.DataFrame(row_accumulator)
    return new_df

In [130]:
col_list = ['file_number','type','status','file_created','final_action','title']
new_df = splitDataFrameList(df,'other_info',col_list)

In [131]:
new_df.drop(columns='other_info',inplace=True)

In [132]:
new_df.tail()

,file_created,file_number,final_action,link,status,title,type
4711,10/30/1992,921221,5/14/2002,https://milwaukee.legistar.com/LegislationDeta...,Placed On File,An ordinance relating to use of public passeng...,Ordinance
4712,7/7/1992,920585,5/19/2000,https://milwaukee.legistar.com/LegislationDeta...,Placed On File,A substitute ordinance relating to apprentices...,Ordinance
4713,2/5/1992,911966,10/30/1998,https://milwaukee.legistar.com/LegislationDeta...,Placed On File,A substitute ordinance relating to architectur...,Ordinance
4714,11/29/1990,901402,6/25/2002,https://milwaukee.legistar.com/LegislationDeta...,Passed,A substitute ordinance relating to the prohibi...,Ordinance
4715,7/28/1987,870858,9/23/2003,https://milwaukee.legistar.com/LegislationDeta...,Placed On File,Substitute ordinance relating to the change in...,Ordinance


In [161]:
new_df.dtypes

file_created    object
file_number     object
final_action    object
link            object
status          object
title           object
type            object
dtype: object

In [133]:
new_df.to_pickle('legistar_meta_info.pkl')

In [77]:
#meta_legis_dict[2] #worked

In [153]:
# def objectIdentificationUsingWaits(self, maxTimeOut, locatorProperties, locatorType = "xpath"):
#         element = None
#         try:
#             WebDriverWait(self.driver, maxTimeOut, ignored_exceptions=[StaleElementReferenceException]).until(
#                 EC.presence_of_element_located((self.getLocatorType(locatorType), locatorProperties)))
#             element = self.driver.find_element(locatorType, locatorProperties)
#         except:
#             print("Exception occurred during object identification.")
#         return element

In [152]:
# from selenium.common.exceptions import StaleElementReferenceException

# test = visit_site(legistar)
# #time.sleep(5)
# select_all_years(test)
# time.sleep(5)
# select_ordinance(test)
# start_search(test)
# time.sleep(3)
# get_max_results(test)
# time.sleep(.5)



# col_names = [0,'type','status','created_date','final_action','title']
# leg_dict = {}
# r_table_body = get_records_table(test)
# print(r_table_body.text)
# print(r_table_body.find_elements_by_class_name('rgRow'))
# #print(r_table_body.find_elements_by_class_name('rgRow')[0].text)
# for row in r_table_body.find_elements_by_class_name('rgRow'):
#     print(row.get_attribute('id'),'id')
#     file_num = row.find_element_by_css_selector('td').find_element_by_css_selector('a').text
#     leg_dict[file_num] = {}
#     try:
#         link = row.find_element_by_id('ctl00_ContentPlaceHolder1_gridMain_ctl00_ctl04_hypFile').get_attribute('href')
#     except:
#         link='no link'
#     leg_dict[file_num]['file_link'] = link
#     print(file_num,'file_num and ', link,' link')
#     for d,col in zip(row.find_elements_by_css_selector('td'),col_names):
#                 if not col:
#                     continue
                
#                 leg_dict[file_num][col] = d.text

In [8]:
#conn = sqlite3.connect('vr_ords.db')
engine = create_engine('sqlite:///vr_ords.db', echo=False)
connection = engine.connect()
#df.to_sql('users', con=engine)

In [9]:
#new_df.to_sql('meta_data', con=engine)

In [10]:
meta_df = pd.read_sql('SELECT * FROM meta_data;', connection, index_col='index')

In [18]:
meta_df.head()

,file_created,file_number,final_action,link,status,title,type
index,,,,,,,
0,2/5/2019,181642,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance relating to the city’s towing dro...,Ordinance
1,2/5/2019,181644,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance relating to domestic partners of ...,Ordinance
2,2/5/2019,181619,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance to further amend the 2019 offices...,Ordinance
3,2/5/2019,181598,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,An ordinance relating to the change in zoning ...,Ordinance
4,2/5/2019,181583,,https://milwaukee.legistar.com/LegislationDeta...,In Committee,A substitute ordinance relating to traffic con...,Ordinance


In [3]:
def check_status(rowDriver):
    status = rowDriver.find_element_by_id('ctl00_ContentPlaceHolder1_lblStatus2').text
    if len(status):
            return status
    else: return False

def check_passed(rowDriver):
    '''
    if ordinance was passed will say Passed
    '''
    if check_status(rowDriver) == 'Passed':
        return 'Passed'
    elif check_status(rowDriver) == 'Placed On File':
        return 'Placed On File - Denied'
    else:
        return False

In [4]:
def get_legis_table(mkeDriver):
    records_table = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_gridLegislation")
    r_table_body = records_table.find_elements_by_class_name("rgRow")
    r_table_body2 = records_table.find_elements_by_class_name("rgAltRow")
    return r_table_body, r_table_body2

In [103]:
from datetime import datetime

date_pattern = '%m/%d/%Y'

def get_date_time(row):
    #print(row)
    return datetime.strptime(row, '%m/%d/%Y')

def get_year(row):
    
    return row.year
#pd.datetime(meta_df[meta_df.file_created!=' '].file_created[1])

In [100]:
meta_df_post_nov2018 = meta_df[meta_df.file_created!= ' ']

In [110]:
meta_df_post_nov2018[pd.isna(meta_df.link)]

,file_created,file_number,final_action,link,status,title,type,date_time,year
index,,,,,,,,,


In [101]:
meta_df_post_nov2018['date_time'] = meta_df_post_nov2018.file_created.apply(get_date_time)
meta_df_post_nov2018['year'] = meta_df_post_nov2018.date_time.apply(get_year)

In [105]:
meta_df_post_nov2018 = meta_df_post_nov2018[meta_df_post_nov2018.date_time > get_date_time('11/08/2018')]

In [113]:
meta_df_post_nov2018.shape

(40, 9)

In [19]:
meta_df.link[0]


'https://milwaukee.legistar.com/LegislationDetail.aspx?ID=3857556&GUID=A3259554-D312-43C3-B3D9-9697F6D567B1&Options=ID|Text|&Search='

In [ ]:
def get_passed

In [ ]:
testDrive = visit_site(meta_df.link.iloc[-2])

In [43]:
meta_df.link.iloc[-2]

'https://milwaukee.legistar.com/LegislationDetail.aspx?ID=145734&GUID=1097BAA1-1B3D-443C-9ABA-2DA545261E52&Options=ID|Text|&Search='

In [12]:
import requests as req
import bs4

records = req.get(meta_df.link.iloc[-2])

soup = bs4.BeautifulSoup(records.content,'html.parser')

In [23]:
print(meta_df.link.iloc[-2])

https://milwaukee.legistar.com/LegislationDetail.aspx?ID=145734&GUID=1097BAA1-1B3D-443C-9ABA-2DA545261E52&Options=ID|Text|&Search=


In [43]:
# def get_legis_table(mkeDriver):
#     records_table = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_gridLegislation")
#     r_table_body = records_table.find_elements_by_class_name("rgRow")
#     r_table_body2 = records_table.find_elements_by_class_name("rgAltRow")
#     return r_table_body, r_table_body2

def get_legis_table_soup(link):
    records = req.get(link)
    soup = bs4.BeautifulSoup(records.content,'html.parser')
    r_table_body = soup.find(id = 'ctl00_ContentPlaceHolder1_gridLegislation').find_all('tr','rgRow')
    r_table_body2 = soup.find(id = 'ctl00_ContentPlaceHolder1_gridLegislation').find_all('tr','rgAltRow')
    return r_table_body, r_table_body2

In [46]:
#WORKED

table_body_test, table_body_test2 = get_legis_table_soup(meta_df.link.iloc[-2])
pre_url = 'https://milwaukee.legistar.com/'
for  ct,j in enumerate(table_body_test):
    for ct2,i in enumerate(j.find_all('td')):
    #print(i.select('a'))
        if ct2 != 6:
            print(i.get_text())
        else:
            #need ot use regex to get url extension and attach it to standard legistar pre-url
            if i.a.has_attr('onclick'):
                print(pre_url + re.search(r"\'{1}.*\',",i.a['onclick'])[0].strip('\'|,'))

7/12/2002
4

CITY CLERK

PUBLISHED
 
 
https://milwaukee.legistar.com/HistoryDetail.aspx?ID=718012&GUID=86E98FD2-FAE6-4E31-A26D-F9AC5FAE9CD5

Meeting details



Not available


6/25/2002
4

COMMON COUNCIL

PASSED
Pass
16:1
https://milwaukee.legistar.com/HistoryDetail.aspx?ID=717850&GUID=7DF3C679-287B-4523-BC2F-5067EB235FA5

Meeting details



Not available


6/13/2002
4

CITY CLERK

DRAFT SUBMITTED
 
 
https://milwaukee.legistar.com/HistoryDetail.aspx?ID=717604&GUID=1E13F7CB-3F66-4F98-8353-BE10AF9D4A29

Meeting details



Not available


6/12/2002
3

PUBLIC WORKS COMMITTEE

RECOMMENDED FOR  PASSAGE AND ASSIGNED
Pass
4:1
https://milwaukee.legistar.com/HistoryDetail.aspx?ID=717217&GUID=4C676835-BB9E-4AAD-AD9F-647C5C75B69A

Meeting details



Not available


6/6/2002
2

PUBLIC WORKS COMMITTEE

HEARING NOTICES SENT
 
 
https://milwaukee.legistar.com/HistoryDetail.aspx?ID=717361&GUID=EAB0D336-13B9-4223-B161-1C33F0580695

Meeting details



Not available


6/6/2002
2

PUBLIC WORKS COMMITTEE


In [33]:
list(table_body_test)

['\n',
 <td class="rgSorted"><font face="Tahoma" size="2">7/12/2002</font></td>,
 <td><font face="Tahoma" size="2">4</font></td>,
 <td nowrap="nowrap"><font face="Tahoma" size="2">
 <a id="ctl00_ContentPlaceHolder1_gridLegislation_ctl00_ctl04_hypBody"><font color="Black" face="Tahoma" size="2">CITY CLERK</font></a>
 </font></td>,
 <td><font face="Tahoma" size="2">PUBLISHED</font></td>,
 <td><font face="Tahoma" size="2"> </font></td>,
 <td><font face="Tahoma" size="2"> </font></td>,
 <td><font face="Tahoma" size="2">
 <a href="#" id="ctl00_ContentPlaceHolder1_gridLegislation_ctl00_ctl04_hypDetails" onclick="radopen('HistoryDetail.aspx?ID=718012&amp;GUID=86E98FD2-FAE6-4E31-A26D-F9AC5FAE9CD5', 'HistoryDetail');return false;"><font color="Blue" face="Tahoma" size="2">Action details</font></a>
 </font></td>,
 <td><font face="Tahoma" size="2">
 <a class="meeting_NoMeeting" id="ctl00_ContentPlaceHolder1_gridLegislation_ctl00_ctl04_hypMeetingDetail"><font color="Gray" face="Tahoma" size="2">Me

In [48]:
def get_passed_dets_link(x):
    '''
    The line item with the vote is PASSED
    '''
    pre_url = 'https://milwaukee.legistar.com/'
    link = str(x)
    #leg_drive = visit_site(link, headless=True,phantom=False)
    #     if not leg_drive:
    #         return None
    if not len(link):
        return None
    test_dict = {}
    r_table_body, r_table_body2 = get_legis_table_soup(link)
    for ct, row in enumerate(r_table_body+r_table_body2):
        test_dict[ct] = {}
        row_elements = row.find_all('td')
        for k,v in zip(['Date','Ver','Action_By','Action','Result','Tally','A_Details','M_Details','Video'], row_elements):
            if k == 'A_Details':
                if v.a.has_attr('onclick'):
                    test_dict[ct][k] = pre_url + re.search(r"\'{1}.*\',",v.a['onclick'])[0].strip('\'|,')
            else:
                test_dict[ct][k] = v.get_text()
    test_dict = {k:row for k, row in test_dict.items() if row['Action'] == 'PASSED'}
    if len(test_dict):
        return test_dict
    else:
        return None
        
        

In [60]:
test_dict

{1: {'Date': '6/25/2002',
  'Ver': '4',
  'Action_By': 'COMMON COUNCIL',
  'Action': 'PASSED',
  'Result': 'Pass',
  'Tally': '16:1',
  'A_Details': 'https://milwaukee.legistar.com/LegislationDetail.aspx?ID=145734&GUID=1097BAA1-1B3D-443C-9ABA-2DA545261E52&Options=ID|Text|&Search=#',
  'M_Details': 'Meeting details',
  'Video': 'Not available'}}

In [50]:
import warnings
warnings.filterwarnings('ignore')
from multiprocessing import Pool
links_dict = {}
start = time.time()
with Pool(processes=10) as pool:
#         for i, row in enumerate(meta_df.iterrows()):
#             #print(row)
#             rowDriver = visit_site(row[1]['link'], True)
#             cv = check_passed(rowDriver)
#             if cv == 'Passed':
#                 rv = get_passed_dets_link(rowDriver)
#                 if rv:
#                     links_dict[i] = rv
    out = pool.map(get_passed_dets_link, meta_df.link)
    end=time.time()

print(end-start)

685.8667962551117


In [55]:
good_rows=[v  for row in out if row !=None for k,v in row.items()]

In [58]:
passed_df = pd.DataFrame(good_rows)

passed_df.to_sql('passed_legislation', con=engine)